# Step 0. Definition

In [ ]:
#import dtnmonitor_v2 as dtn
import dtn_module.dtnmonitor_v3  as DtnModule
import os,sys
import getpass
import subprocess

dtn = DtnModule.DTNMonitor()

user=getpass.getuser()

DtnModule.show_interface()

DtnModule.show_nvme()

## this is necessary to show exec info in notebook
def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)   


# Step 1. Parameter Setting

In [ ]:
########## Necessary
###  SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
dtn.interface = 'all'
#dtn.interface = 'all'


### P2. SELECT THE INPUT FILE , 
## used in Step 1 : file IO read file
file_read_fio = "/data/disk0/sc17/fftest"


### P3. NUTTCP SERVER IP 
## used in Step 2,3 : input nuttcp server's ip
nuttcp_server="192.168.63.52"

### P4. NUTTCP THREADS
## used in Step 2,3 :input threads number
count = 8
  

# Step 1. Test file IO

In [ ]:
# setup fio parameters
# first, filename is assigned with existed and >1GB file path 
randrepeat=" --randrepeat=0" #--randrepeat=0 
bs=" --bs=1M" #--bs=1M 
ioengine=" --ioengine=sync "#--ioengine=sync 
tuntime=" --runtime=30" #--runtime=30 
iodepth=" --iodepth=32"#--iodepth=32 
name=" --name=drive0"#--name=drive0 
size=" --size=100G "#--size=100G 
filename=" --filename="+file_read_fio #--filename=filename
numjobs=" --numjobs="+str(count) 



In [ ]:
## method 1 : launching subprocess
#for i in range(count):
#    filename=" --filename="+file_read_fio #--filename=filename
#    cmd="fio --thread --rw=read --readonly --norandommap --group_reporting --time_based "
#    command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + filename +" & "
#    print(command)
#    os.system(command)

## method 2 : multithreads by parameters 
filename=" --filename="+file_read_fio #--filename=filename
cmd="sleep 3; fio --thread --rw=read --readonly --norandommap --group_reporting --time_based"
#command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + numjobs + filename 
command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + filename 
print(command)

In [ ]:

os.system(command)

dtn.run_monitor(timeout=60)


# Step 2. Test network transfer by memory

In [ ]:
command = ""
for i in range(count):
     command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
#    if i != (count-1):
#        command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
#    else: 
#        command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
        
#print(command)
os.system(command)
dtn.run_monitor(timeout=60)



# Step 3. Test network transfer by disk

In [ ]:
for i in range(count):
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5300"+str(i)+" -p 5310"+str(i)+ " "+ nuttcp_server +" < "+ file_read_fio +" & "
    #print(command)
    os.system(command)
    #dtn.exec_sysout(command)


dtn.run_monitor(timeout=60)